In [1]:
# General imports
import numpy as np
import sys
basepath = "../"
sys.path.insert(0, basepath)

In [2]:
# Stuff for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams["font.size"]=18
mpl.rcParams["font.family"]="sans-serif"
mpl.rcParams["text.usetex"]=True
mpl.rc('text.latex', preamble=r'\usepackage{amsmath}')

In [3]:
#  Load the 5BZCat catalog
from lib.data import load_5bzcat, catbins, load_hotspots
dpath = "/Users/chiarabellenghi/PointSource/blazar_hotspots_correlation/data/"

cat = load_5bzcat(dpath)

# Load the hotspot list
hotspots = load_hotspots(dpath, hemisphere='south')

ModuleNotFoundError: No module named 'lib'

In [ ]:
# Import stuff for the correlation analysis
from lib.correlation import Correlator, EventSample

In [ ]:
# Define that hypothese to test in terms of minimum pValue threshold and association radius
# Here we test the same hypotheses that are tested in Sara Buson et al 2022 ApJL 933 L43
sigmas = np.linspace(0.4, 0.7, 7)
pVal_thrs = np.linspace(3.5, 4.5, 3)

In [ ]:
# Initialize the Correlator object and the sample of hotspots to analyse.
# Note that catbins is useless for our analysis but the framework still requires it.
C = Correlator(cat, bins=catbins[-1], dec_range=(np.radians(-85),np.radians(-5)), mlat=np.radians(10))
hs_sample = EventSample(hotspots)

# Loop over all the hypotheses for the analysis.
fract_associated = {}
for thr in sorted(pVal_thrs):
    C.add_primary('hotspots', hs_sample)
    C.logpVal_thr = thr
    fract_associated[thr] = []
    for sigma in sigmas:
        print("-------------------------------------------------")
        print(f"Analysis parameters: L_min = {thr}, r_assoc = {sigma:.2f}°")
        print("-------------------------------------------------")
        
        hs_sample.add_sigma_field(np.radians(sigma))
        
        print("*******************************************************************")
        print(
            "Fraction of hotspots having at least one counterpart in the 5BZCat: "
            f"{C()[0]}/{C.primaries['hotspots'].N}"
        )
        
        hotspots_mask, counterparts_mask = C.get_counterparts(catbins[-1])
        fract_associated[thr].append(C()[0]/C.primaries['hotspots'].N * 100)
        
        print()
        print()

In [ ]:
for thr in sorted(pVal_thrs):
    plt.errorbar(sigmas, fract_associated[thr], fmt="o", lw=2, ls="-", label=f"$L_{{\mathrm{{min}}}}={thr}$")

plt.xlabel("$r_{\mathrm{assoc}}$")
plt.ylabel("Associated hotspots [\%]", labelpad=15)
plt.legend(ncol=2, frameon=False, loc='lower left', bbox_to_anchor=(0.0,1.))